In [ ]:
from gurobipy import *
import random
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
import pickle

The dataset used for this case study are confidential, instead we share a template of the csv file (see lji_template.csv)

In [ ]:
dat = pd.read_csv('*.csv', sep=';')

Define variables for the problem using the data

In [ ]:
V = dat.shape[0]
Q = np.log2(dat.iloc[:,12]*dat.iloc[:,10])
Cq = np.ones(V)
num_samples = 1000
# define proportion of trafficking as a random variable
s_mean = dat.iloc[:,3]/dat.iloc[:,2]
s_mean[10]=.31
s_mean[s_mean==0]=.05
mar = .1
s=np.minimum(np.maximum(np.random.triangular(left=s_mean-mar, mode=s_mean, right=s_mean+mar, size=(num_samples, len(s_mean))),0),1)

S1 = [Q*(1-s[j]) for j in range(num_samples)]
S2 = [Q*s[j] for j in range(num_samples)]

num_devices = V
ytpr = (dat.iloc[:,5:10] * [.1,.3,.5,.7,.9]).sum(axis=1)/dat.iloc[:,2]

In [ ]:
tnr = 0.5
ytnr = tnr * np.ones(V)

Compute coefficients for the optimization problem

In [ ]:
coeff2 = [[a*b for a,b in zip(S2[j],ytpr)] for j in range(num_samples)]
coeff3 = [[a*b for a,b in zip(S1[j],ytnr)] for j in range(num_samples)]
coq = [S1[j]*(1-ytnr)+S2[j]*(1-ytpr) for j in range(num_samples)]

DTRAP-DIFFRES for LJI

In [ ]:
alloc_lji = dat.iloc[:,11]
B = sum(dat.iloc[:,11])
K = 55
L = 5*K
H1 =  np.random.uniform(.5, 1, B)

In [ ]:
m = Model()
m.ModelSense = 1  # minimize
# Add variables
x = m.addVars(V, B, vtype=GRB.BINARY, name='x')
error = m.addVars(num_samples, obj=1.0/num_samples, name='error')
# m.setObjective(quicksum(coq[i]*x[i] for i in range(V)), GRB.MINIMIZE)
# Set constraints
# m.addConstr(quicksum(Cq[i]*x[i,k] for i in range(V) for k in range(num_devices)) <= B, name="resources")
m.addConstrs((quicksum(coeff2[j][i]*H1[k]*x[i,k] for i in range(V) for k in range(B)) >= K for j in range(num_samples)), name="HT")
m.addConstrs((quicksum(coeff3[j][i]*H1[k]*x[i,k] for i in range(V) for k in range(B)) <= L for j in range(num_samples)), name="NHT")
m.addConstrs((error[j] == quicksum(coq[j][i]*H1[k]*x[i,k] for i in range(V) for k in range(B)) for j in range(num_samples)), name='error')
# additional constraints for DTRAP-DIFFRES
m.addConstrs((quicksum(x[i,k] for i in range(V)) == 1 for k in range(B)), name="feas1")
m.addConstrs((quicksum(x[i,k] for k in range(B)) <= 10 for i in range(V)), name="feas2")
m.addConstrs((quicksum(x[i,k] for k in range(B)) >= 1 for i in range(V)), name="feas3")
m.update()

In [ ]:
m.optimize()

In [ ]:
sol = [v.X for k,v in x.items()]
obj_vals = [v.X for k,v in error.items()]
k_vals = [quicksum(coeff2[j][i]*H1[k]*x[i,k] for i in range(V) for k in range(B)).getValue() for j in range(num_samples)]
l_vals = [quicksum(coeff3[j][i]*H1[k]*x[i,k] for i in range(V) for k in range(B)).getValue() for j in range(num_samples)]

Save the results and data

In [ ]:
with open('lji_diffres.pkl', 'wb') as f:
    pickle.dump([obj_vals, k_vals, l_vals, sol, H1], f)

In [ ]:
sol1 = np.array(sol)
sol1 = sol1.reshape(V,B)

In [ ]:
plt.rcParams['font.size'] = '12'
for i in range(V):
    tmp1 = sum(sol1[i,:])
    _ = plt.scatter((i+1)*np.ones(int(tmp1)), H1[sol1[i,:]==1], c='blue', alpha=.5, marker='.')
    _ = plt.text(x=(i+1), y=np.max(H1[sol1[i,:]==1]), s=int(tmp1))
_ = plt.xticks(list(range(1,V+1)),list(range(1,V+1)))
_ = plt.xlabel('Stations', fontsize=14)
_ = plt.ylabel('Resource viability', fontsize=14)
plt.savefig('lji_diffres.pdf', bbox_inches='tight', transparent=True)